In [ ]:
import os, logging,math
from collections import deque

from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn import metrics

import datatable as dt
from datatable import f

import mxnet as mx
from mxnet import nd, autograd, gluon
from mxnet.gluon import nn


In [ ]:
### the utils functions
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args


# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class GroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_size : int, default=None
        Maximum size for a single training set.
    Examples
    --------
    >>> import numpy as np
    >>> from sklearn.model_selection import GroupTimeSeriesSplit
    >>> groups = np.array(['a', 'a', 'a', 'a', 'a', 'a',\
                           'b', 'b', 'b', 'b', 'b',\
                           'c', 'c', 'c', 'c',\
                           'd', 'd', 'd'])
    >>> gtss = GroupTimeSeriesSplit(n_splits=3)
    >>> for train_idx, test_idx in gtss.split(groups, groups=groups):
    ...     print("TRAIN:", train_idx, "TEST:", test_idx)
    ...     print("TRAIN GROUP:", groups[train_idx],\
                  "TEST GROUP:", groups[test_idx])
    TRAIN: [0, 1, 2, 3, 4, 5] TEST: [6, 7, 8, 9, 10]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a']\
    TEST GROUP: ['b' 'b' 'b' 'b' 'b']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] TEST: [11, 12, 13, 14]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b']\
    TEST GROUP: ['c' 'c' 'c' 'c']
    TRAIN: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]\
    TEST: [15, 16, 17]
    TRAIN GROUP: ['a' 'a' 'a' 'a' 'a' 'a' 'b' 'b' 'b' 'b' 'b' 'c' 'c' 'c' 'c']\
    TEST GROUP: ['d' 'd' 'd']
    """
    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_size=None
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_size = max_train_size

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))
        group_test_size = n_groups // n_folds
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []
            for train_group_idx in unique_groups[:group_test_start]:
                train_array_tmp = group_dict[train_group_idx]
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)
            train_end = train_array.size
            if self.max_train_size and self.max_train_size < train_end:
                train_array = train_array[train_end -
                                          self.max_train_size:train_end]
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)
            yield [int(i) for i in train_array], [int(i) for i in test_array]

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]


In [ ]:

class MLPModel(nn.Block):
    def __init__(self, hiddenList, dropoutList,actType='relu', **kwargs):
        super(MLPModel, self).__init__(**kwargs)
        assert len(hiddenList) == len(dropoutList)
        assert actType in ['relu','tanh','softrelu']
        self.net=nn.Sequential()
        with self.name_scope():
            self.embedding1 = nn.Embedding(2,2)
            for nHidden, dropout in zip(hiddenList, dropoutList):
                self.net.add(nn.Dense(nHidden, activation=actType))
                self.net.add(nn.Dropout(dropout))
            self.Resp = nn.Dense(1, activation='sigmoid')
            self.Resp1 = nn.Dense(1, activation='sigmoid')
            self.Resp2 = nn.Dense(1, activation='sigmoid')
            self.Resp3 = nn.Dense(1, activation='sigmoid')
            self.Resp4 = nn.Dense(1, activation='sigmoid')

    def forward(self,  X):
        embed1 = self.embedding1(X[:,0])
        embedConcat = nd.concat(embed1,X[:,1:])
        tmpOutput =  self.net(embedConcat)
        ##The multiple output
        resp =  self.Resp(tmpOutput)
        resp1 =  self.Resp1(tmpOutput)
        resp2 =  self.Resp2(tmpOutput)
        resp3 =  self.Resp3(tmpOutput)
        resp4 =  self.Resp4(tmpOutput)
        return resp[:,0],resp1[:,0],resp2[:,0],resp3[:,0],resp4[:,0]

In [ ]:

"""
The trainer framework for nn models
"""
def utilityScoreBincount(date, weight, resp, action):
    countI = len(np.unique(date))
    Pi = np.bincount(date,weight*resp*action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi**2)) * np.sqrt(250/countI)
    u = np.clip(t,0,6)*np.sum(Pi)
    return u


class MLPTrainer(object):
    def __init__(self, nnModel, modelCtx, dataCtx):
        self.modelCtx = modelCtx
        self.dataCtx = dataCtx
        self.model =  nnModel

    def saveCheckpoint(self,nnModel, savePath, mark, metric):
        if not os.path.exists(savePath):
            os.makedirs(savePath)
        filename = os.path.join(savePath, "mark_{:s}_metrics_{:.3f}".format(mark, metric))
        filename +='.param'
        nnModel.save_parameters(filename)

    def predict(self, testX, batchSize=2000):
        predDtSize = testX.shape[0]
        # if the test dataset is small
        if(predDtSize<=batchSize):
            return self.model(nd.array(testX,dtype='float32', ctx=self.dataCtx))[0]
        # if the test dataset is large(to prevent memory allocation error of mxnet)
        blockSize = math.ceil(predDtSize // batchSize)+1
        predResult = self.model(nd.array(testX[0:batchSize,:],dtype='float32', ctx=self.dataCtx))[0]
        for i in range(1, blockSize):
            subStartIndex = i*batchSize
            subEndIndex = min(subStartIndex+batchSize, predDtSize)
            blockPred = self.model(nd.array(testX[subStartIndex:subEndIndex,:],dtype='float32', ctx=self.dataCtx))[0]
            predResult = nd.concat(predResult, blockPred, dim=0)
        return predResult
            
    def basicEvaluator(self, testX, testY, testW, testD,  lossFunc, optThreshold,  metric):
        assert metric in set(['auc','utility','both'])
    
        loss, auc, utility = [0.0, 0.0, 0.0]

        pred =self.predict(testX)
        npPred = pred.asnumpy()
        
        predAction = np.where(npPred>=optThreshold, 1, 0)
        print(sum(predAction>0.5))
        label = testY[:,4]>0
        # The loss
        loss = nd.mean(lossFunc(pred, nd.array(label, ctx=self.dataCtx)))[0].asscalar()
        # The auc
        fpr, tpr, thresholds = metrics.roc_curve(label, npPred)
        auc = metrics.auc(fpr, tpr)
        # The utility 
        utility = utilityScoreBincount(testD, testW, testY[:,4], predAction)
        return loss, auc, utility

    def fit(self,mark, paramsDict, trainX, trainY, trainW, testX=None, testY=None, testW=None, testD=None):
        """
        The parameters list:
            esEpochs: the early-stopping epoch: -1, non early stopping
        """
        epochs = paramsDict['epochs']
        esEpochs = paramsDict['esEpochs']
        evalCriteria = paramsDict['evalCriteria']

        batchSize = paramsDict['batchSize']
        learningRate = paramsDict['learningRate']
        sampleRate = paramsDict['sampleRate']
        smoothingAlpha = paramsDict['smoothingAlpha']

        lossFunc = paramsDict['lossFunc']
        optimizer = paramsDict['optimizer']
        initializer = paramsDict['initializer']
        ### The model initialization
        self.model.collect_params().initialize(initializer, ctx=self.modelCtx)
        ### The trainer
        trainer = gluon.Trainer(self.model.collect_params(), optimizer=optimizer, optimizer_params={'learning_rate': learningRate})

        ## 
        nSamples = trainX.shape[0]
        nBatch = int(nSamples / batchSize)
        maxTrainingSample = nSamples*sampleRate

        # Keep the metrics history
        history = dict()
        lossTrainSeq = []
        lossTestSeq = []

        # The early stopping framework
        bestValidMetric = 999999. if evalCriteria =='min' else 0.
        if(esEpochs > 0):
            modelDeque = deque()
            evalMetricDeque= deque()

        for e in tqdm(range(epochs), desc='epochs'):
            cumLoss = 0.
            cumSamples = 0.
            trainIter = gluon.data.DataLoader(gluon.data.ArrayDataset(trainX, trainY), batch_size=batchSize, shuffle=True)
            for  data, allLabel in trainIter:
                data = nd.array(data, dtype='float32', ctx=self.dataCtx)
                allLabel = nd.array(allLabel>0,dtype='float32',ctx=self.dataCtx)
                label = nd.array(nd.abs(allLabel-smoothingAlpha),dtype='float32', ctx=self.dataCtx)
                with autograd.record():
                    resPred,resPred1, resPred2,resPred3, resPred4  = self.model(data)
                    resloss = lossFunc(resPred, label[:,4])
                    resloss1 = lossFunc(resPred1, label[:,0])
                    resloss2 = lossFunc(resPred2, label[:,1])
                    resloss3 = lossFunc(resPred3, label[:,2])
                    resloss4 = lossFunc(resPred4, label[:,3])
                    loss = 8*resloss+resloss1+resloss2+resloss3+resloss4
                loss.backward()
                trainer.step(batchSize)
                batchLoss = nd.sum(loss).asscalar()
                batchAvgLoss = batchLoss / data.shape[0]
                cumLoss += batchLoss

                cumSamples += batchSize
                if(cumSamples>maxTrainingSample): break
            #sampling

            logging.info("Epoch %s / %s. Loss: %s." % (e + 1, epochs, cumLoss / nSamples))
            print("Epoch %s / %s. Training Loss: %s." % (e + 1, epochs, cumLoss / nSamples))
            lossTrainSeq.append(cumLoss/nSamples)
            if not testX is None:
                testLoss, testAuc,testUtility = self.basicEvaluator(testX, testY, testW,testD, lossFunc,0.5, 'auc')
                print("Epoch %s / %s. Testing loss: %s. Testing utility: %s. Testing AUC: %s. Best AUC: %s" % (e + 1, epochs, testLoss, testUtility, testAuc, bestValidMetric))
                ###Save the model
                #self.saveCheckpoint('Params',mark, e)
                ### The early stopping framework
                if(testAuc > bestValidMetric):
                    modelDeque.clear()
                    modelDeque.append(self.model)

                    evalMetricDeque.clear()
                    evalMetricDeque.append(testAuc)
                    ## update the best metrics
                    bestValidMetric = testAuc

                elif len(modelDeque) < esEpochs:
                    modelDeque.append(self.model)
                    evalMetricDeque.append(testAuc)
                else:
                    break
        bestModel = modelDeque.popleft()
        bestMetric = evalMetricDeque.popleft()
        #self.saveCheckpoint(bestModel, 'Params',mark, bestMetric)

        return bestModel

In [ ]:
"""
Load the dataset and preprocessing
"""
dataDirPath = "/kaggle/input/jane-street-market-prediction/"
train = dt.fread(dataDirPath + 'train.csv', na_strings = ['NA','NULL','NaN','\\N'])
# select part of data for baseline testing
train = train[(f.date>85) & (f.weight>0),:]
# add classification indicator
train[:,["action1","action2","action3","action4","action"]] = train[:, [f.resp_1>0, f.resp_2>0, f.resp_3>0, f.resp_4>0, f.resp>0]]
train[["action1","action2","action3","action4","action"]] = dt.int32
# The features
features = [c for c in  train.names if 'feature' in c]
resps = [c for c in train.names if 'resp' in c]
actions = [c for c in train.names if 'action' in c]

# The imputation mean
meanImputeValues = train[:, 8:137].mean().to_numpy()[0,:]
#np.savetxt('meanImputation.npy', meanImputeValues,delimiter=',')
#meanImputeValues = np.loadtxt('meanImputation.npy')

### The basic imputation
train = train.to_pandas()
train = train.astype({c: np.float32 for c, t in train.dtypes.items() if t == np.float64})
train = train.fillna(train.mean())

train.loc[train['feature_0'] ==-1, 'feature_0'] = 0

In [ ]:
nSplits = 5
groupGap = 5

cv = PurgedGroupTimeSeriesSplit(
    n_splits=5,
    max_train_group_size=np.inf,
    group_gap=0,
    max_test_group_size=np.inf
)



foldList = []
trainXList = []; trainYList = []; trainWList = []
validXList = []; validYList = []; validWList =[];validDList = [];

for fold, (tr, te) in enumerate(cv.split(train['resp'].values, train['resp'].values, train['date'].values)):
    if fold != 4: continue
    subTrain, subValid = train.loc[tr,], train.loc[te,]

    trainX, trainY, trainW = subTrain.loc[tr, features].values, subTrain.loc[tr, resps].values, subTrain.loc[tr, 'weight'].values
    validX, validY, validW,validD = subValid.loc[te, features].values, subValid.loc[te, resps].values, subValid.loc[te, 'weight'].values, subValid.loc[te, 'date'].values

    foldList.append(fold)
    trainXList.append(trainX);  trainYList.append(trainY);  trainWList.append(trainW)
    validXList.append(validX);  validYList.append(validY);  validWList.append(validW); validDList.append(validD)

In [ ]:


hiddenList = [128, 64,64]
dropoutList = [0.2, 0.3,0.3]

baseMLP = MLPModel(hiddenList,dropoutList)
modelPrefix = 'MLP_{}d{}l.params'.format(hiddenList[0], len(hiddenList))

modelCtx = mx.gpu() if mx.context.num_gpus() else mx.cpu()
dataCtx = mx.gpu() if mx.context.num_gpus() else mx.cpu()

mlpTrainer = MLPTrainer(baseMLP, dataCtx, modelCtx)

""""
Define the trainer
"""
from mxnet.gluon.loss import SigmoidBinaryCrossEntropyLoss

epochs = 50
esEpochs = 3
evalCriteria = 'max'

batchSize = 128*8
learningRate = 0.001
sampleRate = 0.8
smoothingAlpha=0.005

initializer = mx.init.Xavier(magnitude=2.24)
optimizer = 'adam';
lossFunc = SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)


trainerParamsList = {'epochs': epochs, 'esEpochs': esEpochs, 'evalCriteria': evalCriteria,
        'batchSize': batchSize, 'learningRate': learningRate, 'sampleRate': sampleRate, 'smoothingAlpha':smoothingAlpha,
                    'initializer': initializer, 'optimizer':optimizer, 'lossFunc': lossFunc}


In [ ]:
"""
The model training
"""
modelList = []
for fold in foldList:
    fold=0
    subMark = "fold"+str(fold)
    trainX, trainY, trainW = np.array(trainXList[fold]), np.array(trainYList[fold]),np.array(trainWList[fold])
    testX, testY, testW,testD = np.array(validXList[fold]), np.array(validYList[fold]), np.array(validWList[fold]), np.array(validDList[fold])
    subFit = mlpTrainer.fit(subMark, trainerParamsList, trainX, trainY, trainW,  testX, testY, testW,testD)
    modelList.append(subFit)
    break

### Submitting

In [ ]:

import janestreet
env = janestreet.make_env()
envIter = env.iter_test()


optThreshold = 0.5 

for (testDf, predDf) in tqdm(envIter):
    if testDf['weight'].item() > 0:
        testDf.loc[testDf['feature_0'] == -1, 'feature_0'] = 0
        xTT = testDf.loc[:, features].values
        if np.isnan(xTT[:, 1:].sum()):
            xTT[:, 1:] = np.nan_to_num(xTT[:, 1:]) + np.isnan(xTT[:, 1:]) * meanImputeValues
        pred = 0.
        pred = subFit(nd.array(xTT,ctx=dataCtx))[0].asnumpy()
        predDf.action = np.where(pred >= optThreshold, 1, 0).astype(int)
    else:
        predDf.action = 0
    env.predict(predDf)
